In [ ]:
%pip install pandas
%pip install nltk
%pip install scikit-learn

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load the datasets
try:
  df_train = pd.read_csv('/content/drive/MyDrive/TUS/Engineering_Project/data/source_files/all_train.tsv', sep='\t', encoding='latin1', low_memory=True)
  df_test = pd.read_csv('/content/drive/MyDrive/TUS/Engineering_Project/data/source_files/all_test_public.tsv', sep='\t', encoding='latin1', low_memory=True)
  df_validate = pd.read_csv('/content/drive/MyDrive/TUS/Engineering_Project/data/source_files/all_validate.tsv', sep='\t', encoding='latin1', low_memory=True)

except Exception as e:
  print(f"An error occurred during data loading: {e}")

try:
  df_submissions_clean = pd.concat([df_train, df_test, df_validate], ignore_index=True)
except Exception as e:
  print(f"An error occurred during concatenation: {e}")

# Save the cleaned DataFrame to a new CSV file
try:
    df_submissions_clean.to_csv('/content/drive/MyDrive/TUS/Engineering_Project/data/concatted_all_data.csv', index=False)
except Exception as e:
    print(f"An error occurred saving to csv: {e}")

In [ ]:
%pip install pandas
%pip install nltk
%pip install scikit-learn

from google.colab import drive # type: ignore
drive.mount('/content/drive', force_remount=True)

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

import io

# Toggle for using static test data
USE_STATIC_DATA = True

# Static test data
static_comments_data = {
    'id': [8035415, 8261647, 8261648, 8261649, 8261650, 8261651, 8261652, 8261653, 8261654, 8261655, 8261656, 8261657, 8261658, 8261659, 8261660, 8261661, 8261662, 8261663, 8261664, 8261665, 8261666, 8261667, 8261668, 8261669, 8261670, 8261671, 8261672, 8261673, 8261674, 8261675, 8261676, 8261677, 8261678, 8261679, 8261680, 8261681, 8261682, 8261683, 8261684, 8241541, 8241542, 8241543, 8241544, 8241545, 8231680, 8231681, 8231682],
    'author': ['cpzjcsn', 'RabbiVolesSolo', 'now_stop_that', '', 'Gwunt', 'Kuritos', 'msbutah', 'HomerJunior', '', '', 'IratePieRater', 'MakesPensDance', 'showmeyourlove', 'Tetragrade', 'corvustock', 'CrispyPudding', 'lifeisjest', 'somestupidname1', 'GoghGirl', 'GoghGirl', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'paulrulez742', 'IratePieRater', 'paulrulez742', '', 'Mobbasta', '', 'DoofusMagnus', 'ELF4000', 'stimpakish', 'phd_dude', ''],
    'body': ["?", ";)", ";)", ";)", ";)", ";)", ";)", "( ͡~ ͜ʖ ͡°)", "[deleted]", "[deleted]", ";)", ";)", ";)", ":^)", ";)", "+‿-", "Some really weird shit pops up in r/all ...", ";)", "^(^-^) ^", "(;", ";)", ";)", ";)", ";)", ";)", ";)", ";)", ";)", ";)", ";)", ";)", ";)", ";)", ";)", ";)", ";)", ";)", "^(^—^) ^", "(;", "\"One of us. One of us.\"", "._.", "•____•", ".▂.", ".-.", ":[]", ":[ ]", "[deleted]"],
    'isTopLevel': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, False, False],
    'parent_id': ['t3_3148se', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't3_3iqltn', 't1_cuiqotf', 't1_cuitm7b', 't1_cuiu4f7', 't1_cuj34lk', 't1_cuj349r', 't1_cuj64dt', 't1_cuj7uzl', 't1_cujjn82', 't1_cujl3rx', 't1_cujarhm', 't1_cujcwje', 't1_cujebjb', 't1_cuivqrs', 't1_cujmqmf', 't1_cujoke2', 't1_cuj7h2o', 't1_cujgz0t', 't1_cujt1t1', 't1_cujj9h0', 't3_53pohv', 't3_53ov0q', 't3_53ov0q', 't3_53ov0q', 't3_53ov0q', 't3_5rpn5l', 't1_dd954mt', 't1_dd962ss'],
    'submission_id': ['3148se', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '3iqltn', '53pohv', '53ov0q', '53ov0q', '53ov0q', '53ov0q', '5rpn5l', '5rpn5l', '5rpn5l'],
    'ups': [3.0, 16.0, 11.0, 9.0, 7.0, 6.0, 7.0, 8.0, 3.0, 3.0, 4.0, 4.0, 6.0, 5.0, 2.0, 2.0, 4.0, 2.0, 1.0, 1.0, 6.0, 7.0, 9.0, 7.0, 5.0, 1.0, 5.0, 1.0, 2.0, 2.0, 3.0, 1.0, 4.0, 2.0, 2.0, 4.0, 1.0, 2.0, 2.0, 1.0, 15.0, 6.0, 5.0, 1.0, 1.0, 1.0, 1.0]
}

static_submissions_data = {
    'Unnamed: 0': [10646, 149720, 744801, 167700, 456042],
    'Unnamed: 0.1': [10646, 149720, 744801, 167700, 456042],
    'Unnamed: 0.1.1': [None, None, None, None, None],
    'author': ['rpilek', 'paulrulez742', 'tinywinner', 'Wizard-ette', 'patho5'],
    'clean_title': ['australian wwi poster', 'sure sit on me and spin', 'this horde of ikea drawer stoppers', 'oo', ':[]'],
    'created_utc': [1427925770.0, 1440776896.0, 1474406090.0, 1474396501.0, 1486071264.0],
    'domain': ['i.imgur.com', 'imgur.com', 'i.reddituploads.com', 'imgur.com', 'i.reddituploads.com'],
    'hasImage': [True, True, True, True, True],
    'id': ['3148se', '3iqltn', '53pohv', '53ov0q', '5rpn5l'],
    'image_url': [
        'https://external-preview.redd.it/4xOStvKJZnwjGoP5gqEnms7etyLq6pRFp7h0JeWKcsU.jpg?width=320&crop=smart&auto=webp&s=cfde9ddf38c4916dacfb5c424d4de3507799c32e',
        'https://external-preview.redd.it/Lc7LWsIEOudLxy-n-TENIY2BshKJBpQbg0Z7NfcKuy4.jpg?width=320&crop=smart&auto=webp&s=9249dd6b86aeb2aafd6f2de07582d3874026d7c5',
        'https://external-preview.redd.it/Z6P_PMpruvYHpjE7ZA72IJxva-MYzxQRSojcQ7MY5xQ.jpg?width=320&crop=smart&auto=webp&s=4f4d318cc07d58ccee4424f225c0f700e57ba917',
        'https://external-preview.redd.it/_nMpG6S5mOrZH53Z190jRm5XACqubYFblL3rw_0v0yc.jpg?width=320&crop=smart&auto=webp&s=8ae0082fcafd2397225b7c9ee3fc8a6e7f987368',
        'https://external-preview.redd.it/GWZg6zeKP0Opjzb1xSixCMyadWkjs2xVOV-B6izCfvQ.jpg?width=320&crop=smart&auto=webp&s=0a424b912b60aafeaeda7ab88d0001d8ce1f3fd9'
    ],
    'linked_submission_id': [None, None, None, None, None],
    'num_comments': [2.0, 39.0, 1.0, 4.0, 3.0],
    'score': [36, 720, 15, 162, 11],
    'subreddit': ['propagandaposters', 'pareidolia', 'pareidolia', 'pareidolia', 'pareidolia'],
    'title': ['Australian WWI Poster. 1914-1918.', 'Sure, sit on me and spin', 'This horde of IKEA drawer stoppers.', 'o_o', ':[]'],
    'upvote_ratio': [0.95, 0.96, 0.74, 0.97, 0.92],
    '2_way_label': [0, 0, 0, 0, 0],
    '3_way_label': [1, 2, 2, 2, 2],
    '6_way_label': [5, 2, 2, 2, 2]
}


# Load the datasets
if USE_STATIC_DATA:
    df_submissions_clean = pd.DataFrame(static_submissions_data)
    df_comments = pd.DataFrame(static_comments_data)
else:
    try:
        df_submissions_clean = pd.read_csv('/content/drive/MyDrive/TUS/Engineering_Project/data/concatted_all_data.csv', encoding='latin1', low_memory=True, on_bad_lines='skip')
    except Exception as e:
        print(f"An error occurred during df_submissions_clean data loading: {e}")

    try:
        df_comments = pd.read_csv('/content/drive/MyDrive/TUS/Engineering_Project/data/source_files/all_comments.tsv', sep='\t', encoding='utf-8', on_bad_lines='warn', low_memory=True)
    except Exception as e:
        print(f"An error occurred during df_comments data loading: {e}")

columns_to_remove = [col for col in df_submissions_clean.columns if col.startswith('Unnamed')]
df_submissions_clean.drop(columns=columns_to_remove, inplace=True)

# Include rows where 'hasImage' column is NOT TRUE/True/true and remove the rest
df_submissions_clean = df_submissions_clean[~df_submissions_clean['hasImage'].astype(str).str.lower().eq('true')]
df_submissions_clean = df_submissions_clean[~df_submissions_clean['subreddit'].str.lower().str.contains('photo')]

df_submissions_clean.drop(columns=["created_utc", "domain", "image_url"], inplace=True)
df_submissions_clean = df_submissions_clean[:40000]
# Check for the presence of 2-way, 3-way, and 6-way columns
print(df_submissions_clean.columns)
required_columns = ['2_way_label', '3_way_label', '6_way_label']
missing_columns = [col for col in required_columns if col not in df_submissions_clean.columns]

if not missing_columns:
    # Drop rows where the 2-way, 3-way, or 6-way columns are not integers
    try:
        df_submissions_clean = df_submissions_clean[df_submissions_clean[required_columns].applymap(lambda x: str(x).isdigit()).all(axis=1)]
        df_submissions_clean[required_columns] = df_submissions_clean[required_columns].astype(int)

        # Filter rows based on value ranges for 2-way, 3-way, and 6-way columns
        df_submissions_clean = df_submissions_clean[
            (df_submissions_clean['2_way_label'].between(0, 2)) &
            (df_submissions_clean['3_way_label'].between(0, 3)) &
            (df_submissions_clean['6_way_label'].between(0, 6))
        ]

        # df_submissions_clean["clean_title_orig"] = df_submissions_clean["clean_title"]

        # Count and print the occurrences of each value in the 6-way column
        count_6_way = df_submissions_clean['6_way_label'].value_counts().sort_index()
        print("Counts of 6-way column values:")
        print(count_6_way)

    except Exception as e:
        print(f"An error occurred during type conversion or filtering: {e}")
else:
    print(f"Missing columns in df_submissions_clean: {', '.join(missing_columns)}")

# Download NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to clean text
def clean_text(text):
    if isinstance(text, float):  # Check if the text is a float (NaN)
        return ''

    # Temporarily replace the special delimiter with a placeholder
    placeholder = "placeholderspecialdelim"
    text = text.replace('|__|', placeholder)

    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z\s' + placeholder + ']', '', text)  # Remove special characters but keep the placeholder
    text = text.lower()  # Convert to lowercase
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])  # Lemmatize and remove stopwords

    # Restore the special delimiter
    text = text.replace(placeholder, '|__|')

    # Limit the length of the text
    max_length = 500  # Set the maximum length for comments and content
    if len(text) > max_length:
        text = text[:max_length] + '...' + '|__|'

    return text


# Create a dictionary to store comments by submission_id
comments_dict = {}
comments_orig_dict = {}

try:
    print("Size of df_comments: " + str(len(df_comments)))

    for i, row in df_comments.iterrows():
        if 'submission_id' in row and pd.notna(row['submission_id']):
            submission_id = row['submission_id']
            comment_body = row['body'] if pd.notna(row['body']) else ''
            comment_body_orig = comment_body
            comment_body = clean_text(comment_body)
            print(f"Processing comment for submission_id: {submission_id} comment: {comment_body}")
            # Check for invalid comment bodies
            if (comment_body in ['NaN', 'deleted', 'removed', ' ' , '[deleted]','[removed]']) or \
              ('post comment without original photoshop' in comment_body.lower() or
                'please post comment photoshop reply' in comment_body.lower() or
                'Unfortunately, your submission has been removed from' in comment_body.lower() or
                'google image search automated response' in comment_body.lower()):
                continue  # Skip this row if the comment is invalid
            # Check for valid submission_id and non-empty comment_body
            elif submission_id != '' and comment_body != '':
                print(f"Adding comment for submission_id: {submission_id} comment: {comment_body}")
                if submission_id in comments_dict:
                    comments_dict[submission_id].append(comment_body)
                    comments_orig_dict[submission_id].append(comment_body_orig)
                else:
                    comments_dict[submission_id] = [comment_body]
                    comments_orig_dict[submission_id] = [comment_body_orig]
    print("Dictionary created successfully")

    # Initialize comments column without cleaning 'clean_title' or 'comments'
    df_submissions_clean['comments'] = df_submissions_clean.apply(
        lambda row: '|__|'.join(comments_dict.get(row['linked_submission_id'], []) + comments_dict.get(row['id'], [])),
        axis=1
    )

    df_submissions_clean['comments_orig'] = df_submissions_clean.apply(
        lambda row: '|__|'.join(comments_orig_dict.get(row['linked_submission_id'], []) + comments_orig_dict.get(row['id'], [])),
        axis=1
    )

    # Remove rows without any comments
    df_submissions_clean = df_submissions_clean[df_submissions_clean['comments'] != '']
    print("Comments successfully added to submissions")

    # Count and print the occurrences of each value in the 6-way column
    count_6_way = df_submissions_clean['6_way_label'].value_counts().sort_index()
    print("Counts of 6-way column values:")
    print(count_6_way)
except Exception as e:
    print(f"An error occurred: {e}")

    # Remove tabs and new line characters from 'comments_orig' and 'title' columns
df_submissions_clean['comments_orig'] = df_submissions_clean['comments_orig'].str.replace('\t', ' ').str.replace('\n', ' ')
df_submissions_clean['title'] = df_submissions_clean['title'].str.replace('\t', ' ').str.replace('\n', ' ')

# Save the cleaned DataFrame to a new CSV file without cleaning 'clean_title' or 'comments'
try:

#    df_limited=df_submissions_clean.head(1000)
    df_submissions_clean.to_csv('/content/drive/MyDrive/TUS/Engineering_Project/data/merged_cleaned_data_v26_NoImage.tsv', sep='\t', index=False)
except Exception as e:
    print(f"An error occurred saving to csv: {e}")